### Query Enhancement – Query Expansion Techniques

In a RAG pipeline, the quality of the query sent to the retriever determines how good the retrieved context is — and therefore, how accurate the LLM’s final answer will be.

That’s where Query Expansion / Enhancement comes in.

#### 🎯 What is Query Enhancement?
Query enhancement refers to techniques used to improve or reformulate the user query to retrieve better, more relevant documents from the knowledge base.
It is especially useful when:

- The original query is short, ambiguous, or under-specified
- You want to broaden the scope to catch synonyms, related phrases, or spelling variants

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

In [2]:
## step1 : Load and split the dataset
loader = TextLoader("langchain_crewai_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)


In [3]:
chunks

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using st

In [4]:
### step 2: Vector Store
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(chunks,embedding_model)

## step 3:MMR Retriever
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5})
retriever


c:\Users\ChitreshKaushik\Documents\RAG_UseCases\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000267DC1F06E0>, search_type='mmr', search_kwargs={'k': 5})

In [5]:
## step 4 : LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model("groq:gemma2-9b-it")

In [14]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.Only return one expanded query .

Original query: "{query}"

Expanded query:
""")

query_expansion_chain=query_expansion_prompt| llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.Only return one expanded query .\n\nOriginal query: "{query}"\n\nExpanded query:\n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000267D8FC9160>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000267D8FC9D30>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [15]:
print(query_expansion_chain.invoke({"query":"Langchain memory"}))

"LangChain memory management techniques, persistent storage options, and best practices for using memory in LangChain applications" 



In [16]:
# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

document_chain=create_stuff_documents_chain(llm=llm,prompt=answer_prompt)

In [17]:
# Step 5: Full RAG pipeline with query expansion
rag_pipeline = (
    RunnableMap({
        "input": lambda x: x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query": x["input"]}))
    })
    | document_chain
)

In [19]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

"{'input': 'Which memory types are compatible with the LangChain framework? Please list the specific memory stores and their functionalities.'}" 


Here's why this expansion is helpful:

* **"Which memory types"**  is a more direct and formal way to ask about supported memory.
* **"compatible with the LangChain framework"** provides crucial context, ensuring the search focuses on LangChain's capabilities.
* **"Please list the specific memory stores and their functionalities"**  requests a clear and detailed answer, improving the likelihood of retrieving relevant and informative documents. 


✅ Answer:
 According to the context, LangChain supports memory modules like **ConversationBufferMemory** and **ConversationSummaryMemory**. 


These modules allow the LLM to remember past conversation turns or summarize long interactions. 



In [20]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))


"{'input': 'CrewAI capabilities? functionality? features?  or  explain how CrewAI agents work?'}" 


Here's why this expansion is helpful:

* **Synonyms:**  "Capabilities," "functionality," and "features" are synonyms for "agents" and provide broader search terms.
* **Technical Terms:**  No additional technical terms are needed as "CrewAI agents" is already specific.
* **Context:** Adding the phrase "or explain how CrewAI agents work?"  encourages the retrieval of documents that go beyond just listing features and delve into the inner workings of CrewAI agents. 


By using these additions, the query is more comprehensive and likely to return relevant and informative results.



In [13]:
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

✅ Answer:
 CrewAI agents are semi-autonomous entities designed to work together in structured workflows. 

Here's a breakdown based on the context:

* **Purposeful:** Each agent has a specific purpose, goal, and set of tools.
* **Collaborative:** They operate within a team structure, with defined roles like researcher, planner, or executor.
* **Task-Oriented:** The framework ensures they stay focused and contribute to the overall crew objective.
* **Specialized:** They excel in multi-step workflows where complex tasks benefit from division of labor.


Let me know if you have any more questions!

